Display the different models performance over each simulation

In [ ]:
import os 
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

custom_params = {"axes.spines.right": False, "axes.spines.top": False, "axes.spines.left": False,
                 "axes.spines.bottom": False, "figure.dpi": 100}
sns.set_theme(style = "whitegrid", rc = custom_params, font_scale = 1.75)

In [ ]:
path = 'results/'

In [ ]:
# Naming for all 
models = {
    'correct': 'Correct', 
    'mm': 'LMM',
    'all': 'All', 
    'outcomes': r'Mis. $y$', 
    'sigma': r'Mis. $\omega$', 
    'nore': r'No $u^\omega$',
    'sigmanore': r'No $u^\omega$ + Mis. $\omega$',
    'melsm_notimeomega': 'No $u^\omega_{age}$',
    'melsm_notime': 'No slopes',
    'gaussian': 'Gaussian', 
    'nonsinus': 'Non sinus',
    'incorrect': r'Uncorrelated ($u^\omega$, $u^y$)'
    }

ordering = ['Correct', 'All', r'Mis. $y$', r'Mis. $\omega$',
            r'No $u^\omega$', r'No $u^\omega$ + Mis. $\omega$', 
            'No $u^\omega_{age}$', 'No slopes', 'Gaussian', 
            'Non sinus',  r'Uncorrelated ($u^\omega$, $u^y$)']

covariates = {
    # Covariates for mean
    'b_age': (r'$\beta^y[Age]$', 0.5),
    'b_albumin': (r'$\beta^y[Albumin]$', 0.5),
    'b_trig': (r'$\beta^y[Trig]$', 0),
    'b_platelet': (r'$\beta^y[Platelet]$', 0),

    'sd_id__Intercept': (r'$\sigma_{y}$', 1),

    # Covariates for variance
    'b_sigma_age': (r'$\beta^{\omega}[Age]$', 0.8),
    'b_sigma_albumin': (r'$\beta^{\omega}[Albumin]$', 0),
    'b_sigma_trig': (r'$\beta^{\omega}[Trig]$', 0.8),
    'b_sigma_platelet': (r'$\beta^{\omega}[Platelet]$', 0),

    'sd_id__sigma_Intercept': (r'$\sigma_{\omega}$', 0.5),
    'sd_id__sigma_age': (r'$\sigma_{\omega}[Age]$', 0.25)
    }

In [ ]:
# Error to analyze
parameter = 'sd_id__sigma_Intercept'

In [ ]:
naming, value = covariates[parameter]

In [ ]:
def evaluate(path, param):
    print(path, ' - ', len(os.listdir(path)))

    estimates, ci, esterror, coverage = {}, {}, {}, {}

    # Evaluate by iterating across folder structure and averaging for each model types
    for replication in os.listdir(path):
        model_path = path + replication + '/'
        for model in os.listdir(model_path):
            if '.csv' not in model: continue

            csv_file = pd.read_csv(model_path + model, index_col = 0)
            model = models[model[:-4]] # Remove .csv extension
            if model not in estimates:
                estimates[model], ci[model], esterror[model], coverage[model] = [], [], [], []

            param_used = param
            if (param == 'b_age') and (param not in csv_file.index):
                # Special case for sinus
                param_used = 'b_sinage'
                
            if param_used in csv_file.index: 
                csv_file = csv_file.loc[param_used]            
                estimates[model].append(csv_file['Estimate'])
                ci[model].append(csv_file['Q97.5'] - csv_file['Q2.5'])
                esterror[model].append(csv_file['Est.Error'])
                coverage[model].append((csv_file['Q97.5'] > value) & (value > csv_file['Q2.5']))

    return estimates, ci, esterror, coverage

In [ ]:
estimates, ci, esterror, coverage = {}, {}, {}, {}
for experiment in os.listdir(path):
    for number in os.listdir(path + experiment):
        # Check if directory
        if experiment in ['Individuals', 'Points', 'Correlated Random Effect']:
            estimates[(experiment, float(number))], ci[(experiment, float(number))], \
            esterror[(experiment, float(number))], coverage[(experiment, float(number))] = evaluate(path + experiment + '/' + number + '/', parameter)
        else:
            estimates[(experiment, -1)], ci[(experiment, -1)], \
            esterror[(experiment, -1)], coverage[(experiment, -1)] = evaluate(path + experiment + '/', parameter)
            break   

estimates, ci, esterror, coverage = pd.DataFrame(estimates), pd.DataFrame(ci), pd.DataFrame(esterror), pd.DataFrame(coverage)
estimates.index.name, ci.index.name, esterror.index.name, coverage.index.name = 'Models', 'Models', 'Models', 'Models'

# Display tables

In [ ]:
# Std of estimates
std_estimate = estimates.map(np.std)

In [ ]:
# Average of std
average_error = esterror.map(np.mean)

In [ ]:
# Average CI
average_ci = ci.map(np.mean)# .map(lambda x: '{:.2f} ({:.2f})'.format(np.mean(x), np.std(x)))

In [ ]:
display = pd.concat({r'$SD$(' + naming + ')': std_estimate, 
                     r'$\bar{\epsilon}$(' + naming + ')': average_error, 
                     r'$\bar{CI}$(' + naming + ')': average_ci}, axis = 1).reindex(ordering)
display.columns = display.columns.reorder_levels([1, 2, 0])
display = display.sort_index(axis=1, level=[0, 1])

for experiment in display.columns.get_level_values(0).unique():
    save = display[experiment].dropna()
    if len(save.columns.get_level_values(0).unique()) == 1:
        save = save.droplevel(0, axis = 1)
    else:
        save = save.stack(level=0)
        save.index = save.index.reorder_levels([1, 0])
    save = save.to_latex(float_format = '{:.3f}'.format, column_format='c' * (len(save.columns) + len(save.index.names)))
    save = re.sub(r'\\cline\{.*?\}', r'\\midrule', save)
    save = re.sub(r'\\multirow\[t\]', r'\\multirow[c]', save).replace('.000000', '').replace('00000', '')
    
    os.makedirs('images/{}'.format(experiment), exist_ok=True)
    with open('images/{}/{}.tex'.format(experiment, parameter), 'w') as f:
        f.write(save)

# Displays error

In [ ]:
for simulation in estimates.columns.get_level_values(0).unique():
	print(simulation)
	shift = 0.025
	if (simulation == 'Individuals' or simulation == 'Points') and (parameter == 'b_age'):
		display = estimates[simulation]
	else:
		display = estimates[simulation][estimates.index != 'LMM']
		
	display = display.dropna()

	if len(display)	== 0:
		print(f'No data for {simulation}')
		continue

	if (simulation == 'Individuals' or simulation == 'Points'):
		display.columns = display.columns.astype(int)

	order = [m for m in models.values() if m in display.index.unique()][::-1]
	hue = simulation if len(display.columns) > 1 else None

	# Transform in long format (do not repeat data, index is uninformative)
	if hue:
		columns = []
		for column in display.columns:
			columns.append(display[column].explode())
		display = pd.concat(columns, axis = 1).melt(var_name = hue, value_name = naming, ignore_index = False).reset_index()
	else:
		display = display.apply(pd.Series.explode).melt(var_name = simulation, value_name = naming, ignore_index = False).reset_index()
		display = display.drop(simulation, axis = 1)	

	ax = sns.boxplot(display, x = naming, y = "Models", order = order, hue = hue, color='tab:blue' if not hue else None, 
				     palette = sns.color_palette() if hue else ['tab:blue']) 

	yticks = ax.get_yticks()
	if simulation in display.columns:
		hue_values = display[simulation].unique()
		summary_values = display.groupby(["Models", simulation]).max()
		# Extract box positions
		box_positions = {}
		for i, m in enumerate(order):
			for j, h in enumerate(hue_values):
					offset = (j - (len(hue_values) - 1) / 2) / len(hue_values) # Adjust for hue groups
					box_positions[(m, h)] = yticks[i] + offset
	else:
		summary_values = display.groupby(["Models"]).max()
		# Extract box positions
		box_positions = {}
		for i, m in enumerate(order):
			box_positions[(m, -1)] = yticks[i] 

	# Annotate at the right end of each box
	for m, h in box_positions.keys():
		val = np.mean(coverage[simulation, h].loc[m])
		if ~np.isnan(val): 
			ax.text(min(display[naming].max(), 2.05), box_positions[(m, h)], f'{val:.0%}', ha='left',
					fontsize=15, va='center', color='black', 
					backgroundcolor='white')

	plt.xlim(max(0, ax.get_xlim()[0]), min(max(display[naming].max() + shift * 3, ax.get_xlim()[1]), 2.05 + shift * 3))
	plt.axvline(value, color = 'tab:red', linestyle = '--')
	if hue:
		plt.legend(title = simulation, loc='center left', bbox_to_anchor=(1.1, 0.5))

	plt.title(simulation)
	os.makedirs('images/{}'.format(simulation), exist_ok=True)
	plt.savefig('images/{}/{}.png'.format(simulation, parameter), bbox_inches='tight')
	plt.show()